# Rewards Allocation Notebook
This goal of this notebook is to offer an easy way to process the outputs of the praise and sourcecred reward systems, calculate a reward distribution and perform an analysis of the results. It uses mock data and should be considered a work-in-progress. 

### Basic Setup
First, we import the relevant libraries, get the Data and set how many tokens we want to distribute

In [1]:
from ipyfilechooser import FileChooser

import pandas as pd 
import numpy as np 
import analytics_toolbox as tb

fc_praise = FileChooser('./exampleFiles')
fc_sourcecred = FileChooser('./exampleFiles')

print("== Please choose the Praise CSV file == ")
display(fc_praise)
print("== Please choose the Sourcecred CSV file == ")
display(fc_sourcecred)


== Please choose the Praise CSV file == 


FileChooser(path='C:\Users\Pablo\Documents\TEC\ownRepo\praise_RewardAnalysis\exampleFiles', filename='', title…

== Please choose the Sourcecred CSV file == 


FileChooser(path='C:\Users\Pablo\Documents\TEC\ownRepo\praise_RewardAnalysis\exampleFiles', filename='', title…

Now that we have selected the files, we can import them for processing. We can also set the total amount of tokens we want to distribute this period.
Tip: Now that the file paths are set, you can safely click on "Cell > Run all below" from here on the menu bar to execute everything :) 

In [26]:
PRAISE_DATA_PATH = fc_praise.selected
SOURCECRED_DATA_PATH = fc_sourcecred.selected
NUMBER_OF_REWARD_TOKENS_TO_DISTRIBUTE = 1000

praise_data = pd.read_csv(PRAISE_DATA_PATH)
sourcecred_data = pd.read_csv(SOURCECRED_DATA_PATH)

## Praise reward allocation
This method allocates the praise rewards in a very straightforward way: It adds the value of all dished praised together, and then assigns to each user their % of the total.

In [27]:
def calc_praise_rewards(praiseData, tokensToDistribute):
    #we discard all we don't need and and aggregate all the praise for each single user
    slimData = praiseData[['TO', 'FINAL QUANT']].groupby(['TO']).agg('sum').reset_index()
    totalPraisePoints = slimData['FINAL QUANT'].sum()

    slimData['PERCENTAGE'] = slimData['FINAL QUANT']/totalPraisePoints
    slimData['TOKEN TO RECEIVE'] = slimData['PERCENTAGE'] * tokensToDistribute
    return slimData

praise_distribution = calc_praise_rewards(praise_data, NUMBER_OF_REWARD_TOKENS_TO_DISTRIBUTE)

## Combining the Datasets

Now that we have the distribution done, we combine it with sourcecred into one table.
But before continuing, we also want to declare some methods for cleaning the data and prepare it. We'll use them later.

In [28]:
#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -removes the '#' and following from discord names
#  -Some renaming and dropping 
def prepare_praise(praise_data):
    praise_data['TO'] = (praise_data['TO'].str.split('#', 1, expand=False).str[0]).str.lower()
    praise_data.rename(columns = {'TO':'IDENTITY'}, inplace = True)
    praise_data = praise_data[['IDENTITY', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    return praise_data

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -Some renaming and dropping 
#  -changing percentages from 0 - 100 to 0.00-1.00
def prepare_sourcecred(sourcecred_data):
    sourcecred_data.rename(columns = {'%':'PERCENTAGE'}, inplace = True)
    sourcecred_data['IDENTITY'] = sourcecred_data['IDENTITY'].str.lower()
    sourcecred_data['PERCENTAGE'] = sourcecred_data['PERCENTAGE'] / 100
    sourcecred_data = sourcecred_data[['IDENTITY', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    return sourcecred_data

Now we are ready to go.

In [29]:
#generates a new table with combined percentages and added token rewards
# ISSUE: We need single ids
def combine_datasets(praise_data, sourcecred_data):
    processed_praise = prepare_praise(praise_data)
    processed_sourcecred = prepare_sourcecred(sourcecred_data)
    combined_dataset = processed_praise.append(processed_sourcecred, ignore_index=True)

    combined_dataset = combined_dataset.groupby(['IDENTITY']).agg('sum').reset_index()
    #since we just added to percentages
    combined_dataset['PERCENTAGE'] = combined_dataset['PERCENTAGE'] / 2


    return combined_dataset


total_period_praise =combine_datasets(praise_distribution, sourcecred_data)
print(total_period_praise)

   IDENTITY  PERCENTAGE  TOKEN TO RECEIVE
0      bot0    0.009964         20.429892
1      bot1    0.006717         28.485249
2     bot10    0.020388         49.943960
3     bot11    0.026847         59.303503
4     bot12    0.007574         23.584987
5     bot13    0.009725         23.008048
6     bot14    0.003857         13.186561
7     bot15    0.017057         47.020848
8     bot16    0.000029          5.896987
9     bot17    0.000622        124.481710
10    bot18    0.004001          9.461443
11    bot19    0.025120         67.937315
12     bot2    0.023092        164.221255
13    bot20    0.019912         40.507378
14    bot21    0.024544         54.788796
15    bot22    0.027926        173.069363
16    bot23    0.000440         88.086243
17    bot24    0.009140         30.412835
18    bot25    0.004620         48.874776
19    bot26    0.007754         30.057541
20    bot27    0.000023          4.514881
21    bot28    0.000031          6.127338
22    bot29    0.000042          8

![title](img/praiseFusion.jpg)

## Result Analysis
Let's dive into some data analysis! We'll use the metrics designed and explained by octopus🐙 for now, starting with:

### Allocation percentages
This table will show us which percentage if the total rewards gets distributed to which top % of users. So "Top 50% -> 0.85" would mean that the top 50% of praisees received 85% of the total rewards 

In [30]:
p_vals = np.array([50,80,90,95,99])
rewards_rp = np.array([tb.resource_percentage(total_period_praise["TOKEN TO RECEIVE"], p) for p in p_vals])

my_rd_index = [("Top " + str(100 - p) +"%") for p in p_vals]
resource_distribution = pd.DataFrame({"Rewards": rewards_rp}, index = my_rd_index)
print(resource_distribution)

          Rewards
Top 50%  0.818541
Top 20%  0.506210
Top 10%  0.332786
Top 5%   0.230886
Top 1%   0.086535


### Gini coefficient
Next we will look at the Gini coefficient. Note that there is some debate if we want to use this metric at all, since it is usually employed to measure wealth distribution, and not compensation.

In [31]:
p_vals = np.array([0, 50, 80])
rewards_gc = np.array([tb.gini_gt_p(np.array(total_period_praise["TOKEN TO RECEIVE"]), p) for p in p_vals])

my_index = ["All", "Top 50%", "Top 20%"]
gini_coefs = pd.DataFrame({"Rewards": rewards_gc}, index = my_index)
print(gini_coefs)

          Rewards
All      0.477233
Top 50%  0.297089
Top 20%  0.214107


### Shannon Entropy

[Shannon Entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) is a concept from communications theory, which is also used in measuring the diversity of a distribution. The formula for calculating Shannon Entropy among $n$ individuals is
    $$\\sum_{k=1}^n -p_k log_2(p_k),$$
where $p_k$ represents the proportion of the resource that user $k$ received.

Here we compare the actual Shannon Entropy with the maximum possible for the dataset, keeping in mind that a Shannon Entropy of 0 would mean one user holds all the rewards


In [32]:
entropies_df = pd.DataFrame(data = {"Rewards" : tb.calc_shannon_entropies(total_period_praise["PERCENTAGE"]) }, index = ["Entropy", "Max Entropy", "% of Max"])
print(entropies_df)

              Rewards
Entropy      5.703592
Max Entropy  5.643856
% of Max     1.010584


### Nakamoto Coefficient
Last but not least, the Nakamoto coefficient. The Nakamato Coefficient is defined as the smallest number of accounts who control at least 50% of the resource. Although its significance relates to the prospect of a 51% attack on a network, which may not be relevant in our context, we can still use it as an intuitive measure of how many individuals received the majority of a resource.

In [33]:
ak_coef_IH = tb.nakamoto_coeff(total_period_praise, "PERCENTAGE")
print(ak_coef_IH)

31
